In [9]:
import pandas as pd
from tabulate import tabulate

# Загрузка данных
df = pd.read_excel('table1.xlsx', sheet_name='Мед. Блок_Мед.жалобы гл.вра...')

# Проверка названий столбцов (выведем все, чтобы убедиться)
print("Доступные столбцы в данных:")
print(df.columns.tolist())

# Убедимся, что используем правильное название столбца с обоснованностью
# Если столбец называется "Претензия обоснована?", используем его
if 'Претензия обоснована?' in df.columns:
    df.rename(columns={'Претензия обоснована?': 'Обоснованность'}, inplace=True)
elif 'Обращение обосновано?' in df.columns:
    df.rename(columns={'Обращение обосновано?': 'Обоснованность'}, inplace=True)
else:
    raise KeyError("Не найден столбец с обоснованностью в данных")

# Очистка значений в столбце "Обоснованность" (удаляем формулы Excel)
df['Обоснованность'] = df['Обоснованность'].replace(r'^=.*$', 'Разбор', regex=True)

# 1. Статистика по виновным подразделениям
print("\n📊 Статистика по виновным подразделениям:")
print(df['Виновное Подразделение/Клиника'].value_counts().dropna())

# 2. Статистика по функциональным подразделениям
print("\n📊 Статистика по функциональным подразделениям:")
print(df['Функциональное подразделение'].value_counts().dropna())

# 3. Статистика по обоснованности
print("\n📊 Статистика по обоснованности обращений:")
print(df['Обоснованность'].value_counts().dropna())

# 4. Статистика по корневым причинам
print("\n📊 Статистика по корневым причинам:")
print(df['Корневая Причина'].value_counts().dropna())

# 5. Сводная таблица: функциональные подразделения × обоснованность
print("\n📊 Сводная таблица: Функциональные подразделения × Обоснованность")
pivot_func = pd.pivot_table(
    df,
    index='Функциональное подразделение',
    columns='Обоснованность',
    aggfunc='size',
    fill_value=0
)
print(tabulate(pivot_func, headers=pivot_func.columns, tablefmt='grid'))

# 6. Сводная таблица: корневые причины × обоснованность
print("\n📊 Сводная таблица: Корневые причины × Обоснованность")
pivot_cause = pd.pivot_table(
    df,
    index='Корневая Причина',
    columns='Обоснованность',
    aggfunc='size',
    fill_value=0
)
print(tabulate(pivot_cause, headers=pivot_cause.columns, tablefmt='grid'))

# 7. Общая статистика
total = len(df)
justified = len(df[df['Обоснованность'] == 'Обосновано'])
unjustified = len(df[df['Обоснованность'] == 'Не обосновано'])
in_review = len(df[df['Обоснованность'].isin(['Разбор', pd.NA, None])])

print("\n📊 Общая статистика:")
print(f"✅ Обоснованных: {justified} ({justified/total:.1%})")
print(f"❌ Не обоснованных: {unjustified} ({unjustified/total:.1%})")
print(f"🔄 В процессе разбора: {in_review} ({in_review/total:.1%})")
print(f"📌 Всего обращений: {total}")

Доступные столбцы в данных:
['(Не изменять) Обращение', '(Не изменять) Контрольная сумма строки', '(Не изменять) Дата изменения', 'Номер обращения', 'Дата и время приема обращения', 'Обращение обосновано?', 'Виновное Подразделение/Клиника', 'Дата отправки в Виновное подразделение ', 'Дата получения ответа от Виновного подразделения', 'Status Reason', 'Дата закрытия', 'Количество дней ответа ответсвенным исполнителем', 'Согласующий сотрудник']

📊 Статистика по виновным подразделениям:
Виновное Подразделение/Клиника
КДЦБ                                           10
КДЦП                                           10
КДЦС                                           10
КДЦМч                                           9
ЕКДЛ                                            4
Краснобогатырская                               4
Новочеремушкинская                              4
Пролетарский проспект                           4
ДК на Пироговской                               4
Департамент централизованной п

c:\Users\1\IDE\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


KeyError: 'Функциональное подразделение'

In [6]:
# Пример, если столбец называется "Обоснование претензии"
df.rename(columns={'Обоснование претензии': 'Обоснованность'}, inplace=True)

In [7]:
with open('анализ_обращений.txt', 'w', encoding='utf-8') as f:
    f.write("📊 Анализ обращений 📊\n\n")
    f.write("1️⃣ Виновные подразделения:\n")
    f.write(str(df['Виновное Подразделение/Клиника'].value_counts().dropna()) + "\n\n")
    f.write("2️⃣ Функциональные подразделения:\n")
    f.write(str(df['Функциональное подразделение'].value_counts().dropna()) + "\n\n")
    f.write("3️⃣ Обоснованность:\n")
    f.write(str(df['Обоснованность'].value_counts().dropna()) + "\n\n")
    f.write("4️⃣ Корневые причины:\n")
    f.write(str(df['Корневая Причина'].value_counts().dropna()) + "\n\n")
    f.write("5️⃣ Сводная таблица (подразделения × обоснованность):\n")
    f.write(tabulate(pivot_func, headers=pivot_func.columns, tablefmt='grid') + "\n\n")
    f.write("6️⃣ Сводная таблица (корневые причины × обоснованность):\n")
    f.write(tabulate(pivot_cause, headers=pivot_cause.columns, tablefmt='grid') + "\n\n")
    f.write("7️⃣ Итоговая статистика:\n")
    f.write(f"✅ Обоснованных: {justified} ({justified/total:.1%})\n")
    f.write(f"❌ Не обоснованных: {unjustified} ({unjustified/total:.1%})\n")
    f.write(f"🔄 В процессе разбора: {in_review} ({in_review/total:.1%})\n")
    f.write(f"📌 Всего обращений: {total}\n")